In [1]:
# ! gdown 1ooVVPxB-tvptgmHlIMMFGV3Cg-IrhbRZ

Downloading...
From (original): https://drive.google.com/uc?id=1ooVVPxB-tvptgmHlIMMFGV3Cg-IrhbRZ
From (redirected): https://drive.google.com/uc?id=1ooVVPxB-tvptgmHlIMMFGV3Cg-IrhbRZ&confirm=t&uuid=67b3c3b5-e9b2-4ff2-ba0a-70677f009208
To: /home/user01/text_encoder_bias/negCLIP.pt
100%|██████████████████████████████████████| 1.82G/1.82G [03:50<00:00, 7.88MB/s]


In [1]:
import torch
import open_clip
from transformers import AlignProcessor, AlignModel, AutoTokenizer
from transformers import AltCLIPModel, AltCLIPProcessor
from transformers import AutoProcessor, BlipModel
from transformers import FlavaProcessor, FlavaForPreTraining, BertTokenizer, FlavaFeatureExtractor
from transformers import ViltProcessor, ViltForImageAndTextRetrieval
import torch.nn.functional as F
import gc
import numpy as np
import pandas as pd
from PIL import Image

import os

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

Device: cuda


In [3]:
MODELS = {
    "open_clip": [
        # ["ViT-B-32", "negCLIP.pt"], first download and add negClip weights in this directory
        ["EVA01-g-14", "laion400m_s11b_b41k"],
        ["EVA02-L-14", "merged2b_s4b_b131k"],
        ["RN50x64", "openai"],
        ["ViT-B-16", "openai"],
        ["ViT-B-32", "openai"],
        ["ViT-L-14", "openai"],
        ["coca_ViT-B-32", "laion2b_s13b_b90k"],
        ["coca_ViT-L-14", "laion2b_s13b_b90k"],
        ],
    "align": ["kakaobrain/align-base"],
    "alt": ["BAAI/AltCLIP"],
    "flava": ["facebook/flava-full"],
    }

In [4]:
# open_clip
def load_open_clip(activities , report_dict ):
    for base_name, pretrained in MODELS["open_clip"]:
        print(base_name, pretrained)
        model, _, preprocess = open_clip.create_model_and_transforms(base_name, pretrained=pretrained)
        model = model.to(device)
        model.eval()
        tokenizer = open_clip.get_tokenizer(base_name)
        for activity in activities:
            tokenized_text = tokenizer(f"A person is {preprocess_activity(activity)}").to(device)
            tokenized_male_text = tokenizer(f"A man is {preprocess_activity(activity)}").to(device)
            tokenized_female_text = tokenizer(f"A woman is {preprocess_activity(activity)}").to(device)
            with torch.no_grad():
                text_features = model.encode_text(tokenized_text)
                text_features_norm = text_features.norm(dim=-1)
                male_text_features = model.encode_text(tokenized_male_text)
                male_text_features_norm = male_text_features.norm(dim=-1)
                female_text_features = model.encode_text(tokenized_female_text)
                female_text_features_norm = female_text_features.norm(dim=-1)
                male_sim = ((text_features @ male_text_features.T) / (text_features_norm * male_text_features_norm)).item()
                female_sim = ((text_features @ female_text_features.T) / (text_features_norm * female_text_features_norm)).item()
                sim_probs = torch.tensor([male_sim, female_sim]).softmax(dim=-1)
                male_sim_prob, female_sim_prob = sim_probs[0].item(), sim_probs[1].item()
            report_dict["model"].append(f"{base_name} {pretrained}")
            report_dict["activity"].append(activity)
            report_dict["male_sim_prob"].append(np.round(male_sim_prob, 3))
            report_dict["female_sim_prob"].append(np.round(female_sim_prob, 3))
        del model
        torch.cuda.empty_cache()
        gc.collect()

In [5]:
def preprocess_activity(activity):
    return activity.replace("_", " ").lower()

def reverse_gender(gender):
    return "man" if gender == "woman" else "woman"

In [6]:
# align
def load_align(activities , report_dict):
    for model_name in MODELS["align"]:
        print(model_name)
        model = AlignModel.from_pretrained(model_name).to(device)
        model.eval()
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        for activity in activities:
            tokenized_text = tokenizer(f"A person is {preprocess_activity(activity)}", padding=True, return_tensors="pt").to(device) 
            tokenized_male_text = tokenizer(f"A man is {preprocess_activity(activity)}", padding=True, return_tensors="pt").to(device)
            tokenized_female_text = tokenizer(f"A woman is {preprocess_activity(activity)}", padding=True, return_tensors="pt").to(device)
            with torch.no_grad():
                text_features = model.get_text_features(**tokenized_text)
                text_features_norm = text_features.norm(dim=-1)
                male_text_features = model.get_text_features(**tokenized_male_text)
                male_text_features_norm = male_text_features.norm(dim=-1)
                female_text_features = model.get_text_features(**tokenized_female_text)
                female_text_features_norm = female_text_features.norm(dim=-1)
                male_sim = ((text_features @ male_text_features.T) / (text_features_norm * male_text_features_norm)).item()
                female_sim = ((text_features @ female_text_features.T) / (text_features_norm * female_text_features_norm)).item()
                sim_probs = torch.tensor([male_sim, female_sim]).softmax(dim=-1)
                male_sim_prob, female_sim_prob = sim_probs[0].item(), sim_probs[1].item()
            report_dict["model"].append(f"{model_name}")
            report_dict["activity"].append(activity)
            report_dict["male_sim_prob"].append(np.round(male_sim_prob, 3))
            report_dict["female_sim_prob"].append(np.round(female_sim_prob, 3))
        del model
        torch.cuda.empty_cache()
        gc.collect()

In [7]:
# alt
def load_alt(activities , report_dict):
    for model_name in MODELS["alt"]:
        print(model_name)
        model = AltCLIPModel.from_pretrained(model_name).to(device)
        model.eval()
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        for activity in activities:
            tokenized_text = tokenizer(f"A person is {preprocess_activity(activity)}", padding=True, return_tensors="pt").to(device) 
            tokenized_male_text = tokenizer(f"A man is {preprocess_activity(activity)}", padding=True, return_tensors="pt").to(device)
            tokenized_female_text = tokenizer(f"A woman is {preprocess_activity(activity)}", padding=True, return_tensors="pt").to(device)
            with torch.no_grad():
                text_features = model.get_text_features(**tokenized_text)
                text_features_norm = text_features.norm(dim=-1)
                male_text_features = model.get_text_features(**tokenized_male_text)
                male_text_features_norm = male_text_features.norm(dim=-1)
                female_text_features = model.get_text_features(**tokenized_female_text)
                female_text_features_norm = female_text_features.norm(dim=-1)
                male_sim = ((text_features @ male_text_features.T) / (text_features_norm * male_text_features_norm)).item()
                female_sim = ((text_features @ female_text_features.T) / (text_features_norm * female_text_features_norm)).item()
                sim_probs = torch.tensor([male_sim, female_sim]).softmax(dim=-1)
                male_sim_prob, female_sim_prob = sim_probs[0].item(), sim_probs[1].item()
            report_dict["model"].append(f"{model_name}")
            report_dict["activity"].append(activity)
            report_dict["male_sim_prob"].append(np.round(male_sim_prob, 3))
            report_dict["female_sim_prob"].append(np.round(female_sim_prob, 3))
        del model
        torch.cuda.empty_cache()
        gc.collect()

In [8]:
# flava
def load_flava(activities , report_dict):
    for model_name in MODELS["flava"]:
        print(model_name)
        model = FlavaForPreTraining.from_pretrained(model_name).to(device)
        model.eval()
        tokenizer = BertTokenizer.from_pretrained(model_name)
        for activity in activities:
            tokenized_text = tokenizer(f"A person is {preprocess_activity(activity)}", padding=True, return_tensors="pt").to(device) 
            tokenized_male_text = tokenizer(f"A man is {preprocess_activity(activity)}", padding=True, return_tensors="pt").to(device)
            tokenized_female_text = tokenizer(f"A woman is {preprocess_activity(activity)}", padding=True, return_tensors="pt").to(device)
            with torch.no_grad():
                text_features = model.flava.get_text_features(**tokenized_text)[:, 0, :]
                text_features_norm = text_features.norm(dim=-1)
                male_text_features = model.flava.get_text_features(**tokenized_male_text)[:, 0, :]
                male_text_features_norm = male_text_features.norm(dim=-1)
                female_text_features = model.flava.get_text_features(**tokenized_female_text)[:, 0, :]
                female_text_features_norm = female_text_features.norm(dim=-1)
                male_sim = ((text_features @ male_text_features.T) / (text_features_norm * male_text_features_norm)).item()
                female_sim = ((text_features @ female_text_features.T) / (text_features_norm * female_text_features_norm)).item()
                sim_probs = torch.tensor([male_sim, female_sim]).softmax(dim=-1)
                male_sim_prob, female_sim_prob = sim_probs[0].item(), sim_probs[1].item()
            report_dict["model"].append(f"{model_name}")
            report_dict["activity"].append(activity)
            report_dict["male_sim_prob"].append(np.round(male_sim_prob, 3))
            report_dict["female_sim_prob"].append(np.round(female_sim_prob, 3))
        del model
        torch.cuda.empty_cache()
        gc.collect()

In [ ]:
for version in range (1 , 4):
    DATASET_BASE_ADDRESS = f"..\\phaze{version}\\Phaze-{version}"
    expected_genders_df = pd.read_csv(f"{DATASET_BASE_ADDRESS}/expected_genders.csv")
    expected_genders = {activity: expected_genders_df.to_dict("list")["gender"][index]
                        for index, activity in enumerate(expected_genders_df.to_dict("list")["activity"])}
    activities = [activity for activity in os.listdir(f"{DATASET_BASE_ADDRESS}/") if activity[0] != "." and activity !='expected_genders.csv']

    report_dict = {"model": [], 
               "activity":[], 
               "male_sim_prob": [], 
               "female_sim_prob": []}
    
    load_open_clip(activities , report_dict)
    load_align(activities , report_dict)
    load_alt(activities , report_dict)
    load_flava(activities , report_dict )

    RESULTS_BASE_ADDRESS = f"Results/phaze{version}"

    if not os.path.exists("results"):
        os.makedirs("results")

    if not os.path.exists(RESULTS_BASE_ADDRESS):
        os.makedirs(RESULTS_BASE_ADDRESS)

    pd.DataFrame(data=report_dict).to_csv(f"{RESULTS_BASE_ADDRESS}/text_encoder_bias.csv", index=False)
